In [40]:
import os
gpu_num = 0 # Use "" para usar a CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# O Colab atualmente não suporta a versão mais recente do ipython.
# Portanto, a pré-visualização não funciona no Colab. No entanto, sempre que possível,
# recomendamos fortemente usar o modo de pré-visualização da cena.

low_resolution = [360,240] # aumente para obter maior qualidade de renderização
medium_resolution = [560,420]
high_resolution = [800,600]

# Configurar o notebook para usar apenas uma GPU e alocar apenas a quantidade de memória necessária
# Para mais detalhes, veja https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Evitar avisos do TensorFlow
tf.get_logger().setLevel('ERROR')
print(tf.__version__)
tf.random.set_seed(1) # Definir uma semente global para a aleatoriedade, garantindo reprodutibilidade


2.15.1


In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
    
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
from sionna.channel import subcarrier_frequencies, cir_to_ofdm_channel
import matplotlib.pyplot as plt

In [42]:
from scipy.interpolate import interp1d
import ast
# Read the Excel file
df = pd.read_excel('output_new_scene.xlsx')

# Extract the desired columns
desired_columns = ['vehid','steps', 'coord']
df = df[desired_columns]

In [43]:
#Pega os ids dos veiculos
unique_vehids = df['vehid'].unique()
#Pega o maior e o menor valor da coluna steps para saber qual a duração da simulação dentro do SUMO
max_steps = df['steps'].max()
min_steps = df['steps'].min()
simulation_duration = max_steps - min_steps + 1

# Array que receberá os objetos com as informações dos veiculos
vehicles = []
# Seleciona o primeiro veiculo para guardar os valores do primeiro e o último step em que ele aparece
vehid_df = df[df['vehid'] == unique_vehids[0]]
max_first_apper = vehid_df['steps'].iloc[0]
min_last_apper = vehid_df['steps'].iloc[-1]

#Esse loop será responsável por encontrar o último step em que um veiculo aparece e o primeiro step em que um veiculo desaparece. Esses valores serão usados para cortar a simulação no SUMO e evitar que o Sionna tente renderizar um veiculo que não está mais na simulação.
for vehid in unique_vehids:
    vehid_df = df[df['vehid'] == vehid]
    # Acha o primeiro Step em que o carro é rendezirado
    first_apper = vehid_df['steps'].iloc[0]
    # Acha o ultimo Step em que o carro é rendezirado
    last_apper = vehid_df['steps'].iloc[-1]
    # Atualiza o primeiro e o ultimo step em que um veiculo aparece
    if first_apper > max_first_apper:
        max_first_apper = first_apper
    if last_apper < min_last_apper:
        min_last_apper = last_apper
# Corta o dataframe para que ele contenha apenas os veiculos que aparecem em todos os steps
df = df[df['steps'] >= max_first_apper]
df = df[df['steps'] <= min_last_apper]

In [44]:
subcarrier_spacing = 60e3
# Number of used subcarriers 
num_used_subcarriers = 66*12
# Numero de RBs * 12 subcarriers
# Numero de RBs é tabelado, ver link abaixo:
# https://www.nrexplained.com/bandwidth
# 47.52MHz de banda

num_ofdm_symbols = 14 + 1 # 14 OFDM symbols + 1 de guarda

#Carrier Frequency
# 28GHz Por padrão da cena do sionna (e também do fr1(Frequency Range 1 - 3gpp) do 5G)
central_frequency = 28e9

In [45]:
# Devido a escolha
rb_interval = num_ofdm_symbols * 1/subcarrier_spacing # Em segundos
rb_interval = rb_interval * 1e3 # Em milisegundos
print("Tempo de cada RB dada as configurações anteriores",rb_interval, "ms")

# Intervalo em TTI de cada passo da simulação
num_of_TTIs = 10 # em TTI

# Calculando o valores de passos necessarios para ter uma resolução desejada
interpolate_steps = int(1/(rb_interval*1e-3*num_of_TTIs))
print(f"Número de passos por segundo é de {interpolate_steps} passos")

Tempo de cada RB dada as configurações anteriores 0.25 ms
Número de passos por segundo é de 400 passos


In [46]:
#Loop percorrendo cada veiculo para montar o objeto com as informacoes

for vehid in unique_vehids:
    coords=[]
    vehid_df = df[df['vehid'] == vehid]
    coords_list = vehid_df['coord'].tolist()
    
    #Transforma a lista de strings em uma lista de tuplas
    coords = [ast.literal_eval(elem) for elem in coords_list]
    x, y = zip(*coords)
    x = np.array(x)
    y = np.array(y)

    # Cria uma função de interpolação linear para
    f = interp1d(x, y)
    
    # Generate new x values: since we want 4000 new points between each pair, calculate the new step
    new_x = np.linspace(x.min(), x.max(), num=len(x) + interpolate_steps * (len(x) - 1))

    # Use the interpolation function to generate new y values
    new_y = f(new_x)
    obj = {
        'vehid': vehid,
        'first_apper': first_apper,
        'last_apper':last_apper,
        'coords': list(zip(new_x, new_y)),
    }
    vehicles.append(obj)

In [47]:
scene = load_scene("./scenes/bm_v2/scene.xml")
#-38.49168,-3.72718,-38.48401,-3.72205

In [48]:
#Define a frequência central da cena
scene.frequency = central_frequency

In [49]:
# Cria um vetor com as frequências dos subportadores
frequencies = subcarrier_frequencies(num_used_subcarriers, subcarrier_spacing) + scene.frequency
# Reduz o número de frequências para o número de subportadores usados para um limite aceitável
frequencies = frequencies[6::12]

# Esse valores são referentes ao cruzamento entre uma posição de referencia do sumo e da cena no blender
x_diff,y_diff = [421.55, 282.3]

In [50]:
scene.tx_array = PlanarArray(
                                num_rows=4,
                                num_cols=4,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="tr38901",
                                polarization="V"
                            )
scene.rx_array = PlanarArray(
                                num_rows=1,
                                num_cols=1,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="iso",
                                polarization="V"
                            )

tx_1 = Transmitter("tx_1", [-314, -82, 65], [0.0, 0.0, 0.0])
tx_2 = Transmitter("tx_2", [7, 117, 8], [0.0, 0.0, 0.0])
scene.add(tx_1)
scene.add(tx_2)
#tx = Transmitter("tx", [-262, 192, 65], [0.0, 0.0, 0.0])
#scene.add(tx)

In [51]:
total_time = min_last_apper - max_first_apper
print(f"O tempo total da simulação é de {total_time} segundos")

O tempo total da simulação é de 7 segundos


In [56]:
time_to_simulate = 0.1 # Tempo de simulação em segundos
steps_to_simulate = int(time_to_simulate * interpolate_steps) # Número de passos a simular
shift_time = 1 # Tempo em segundos para deslocar a simulação
shift_steps_interpol= int(shift_time * interpolate_steps) # Número de passos a deslocar

total_steps_interpol = shift_steps_interpol + steps_to_simulate # Número total de passos que vão ser simulados

# Filtra o intervalo no dataframe para pegar apenas os passos que serão simulados
for vehicle in vehicles: 
    vehicle['coords'] = np.array(vehicle['coords'][shift_steps_interpol:total_steps_interpol])

In [65]:
import numpy as np

# Inicializa h_simulate como um tensor vazio
h_simulate = None

for interpol_step in range(steps_to_simulate):
    for freq_index, veh in enumerate(vehicles):
        # Pega as coordenadas do veículo
        x1, y1 = veh['coords'][interpol_step]
        # Valores que eu calculo para a cena diff do sumo
        x1, y1 = x1 - x_diff, y1 - y_diff
        # Adiciona o veículo na cena caso seja o seu primeiro step
        if interpol_step == 41:
            scene.add(Receiver(name=veh["vehid"], position=[x1, y1, 1.5]))
        else:
            scene.receivers[veh["vehid"]].position = [x1, y1, 1.5]

    paths = scene.compute_paths(diffraction=True)
    paths.normalize_delays = False
    
    # Calculando o CIR para o usuário e o delay dos caminhos
    a, tau = paths.cir()
    
    # Seleciona o índice do batch e do time step
    batch_index = 0
    time_step_index = 0
    h_new = cir_to_ofdm_channel(frequencies, a, tau)[batch_index, :, 0, :, :, time_step_index]
    
    # Adiciona uma nova dimensão e concatena
    if h_simulate is None:
        h_simulate = np.expand_dims(h_new, axis=0)
    else:
        h_new_expanded = np.expand_dims(h_new, axis=0)
        h_simulate = np.concatenate((h_simulate, h_new_expanded), axis=0)

# Agora h_simulate é um tensor com uma nova dimensão para cada interpol_step

In [66]:
# Redimensiona h_simulate para as novas dimensões
h_simulate_reshaped = np.transpose(h_simulate, (4, 1, 0, 3, 2))


In [81]:
import h5py

with h5py.File("coordinates.h5", mode="w") as f:
    f.attrs["UE Names"] = list(unique_vehids)
    f.attrs["BS Names"] = list(scene.transmitters.keys())
    f.attrs["Time interval"] = rb_interval*num_of_TTIs*1e-3 # Em segundos
    for vehicle in vehicles:
        x, y = zip(*vehicle["coords"])
        x = np.array(x)
        x = x - x_diff
        y = np.array(y)
        y = y - y_diff
        ue_name = vehicle['vehid']
        z = np.linspace(1.5,1.5,len(x))
        df_coords = pd.DataFrame({"x": x, "y": y, "z": z})
        
        f.create_dataset(f"{ue_name}_coordinates", data=df_coords.to_records(index=False))
        
    for tx in scene.transmitters.keys():
        tx_position = np.array(scene.transmitters[tx].position)
        bs_position = pd.DataFrame({"x": [tx_position[0]], "y": [tx_position[1]], "z": [tx_position[2]]})
        f.create_dataset(f"BS_coordinate_{tx}", data=bs_position.to_records(index=False).reshape(()))

    f.attrs["minX"] = -scene.size[0]/2
    f.attrs["maxX"] = scene.size[0]/2
    f.attrs["minY"] = -scene.size[1]/2
    f.attrs["maxY"] = scene.size[1]/2
    
    num_of_ue_antennas = len(scene.transmitters.keys())
    channel_dataset = f.create_group('BS Channels to')
    channel_dataset.attrs["Frequencies"] = frequencies
    
    for freq_index in range(len(frequencies)):
        freq_group = channel_dataset.create_group(f'Freq{freq_index}')
        for veh_index in range(len(vehicles)):
            veh_name = vehicles[veh_index]['vehid']
            veh_group = freq_group.create_group(veh_name)
            veh_group.create_dataset('h', data=h_simulate_reshaped[freq_index,veh_index])